
# AMBRS × CAMP demo — PPE-style setup (H2SO4 example)

This version **builds the PPE using the AMBRS tools exactly like your demo**:
- Define species and a 4-mode modal size specification
- Build `EnsembleSpecification` and sample with `ambrs.lhs(...)`
- Create model inputs with `create_inputs(...)`
- Run baseline vs CAMP for MAM4 and PartMC
- Provide parity + conservation checks (no plotting here)


In [1]:

# -----------------------------------------------------------
# Import modules
# -----------------------------------------------------------
import os
import logging
from math import log10
from pathlib import Path

import numpy as np
import scipy.stats as stats

import ambrs
from ambrs import mam4, partmc
from ambrs.runners import PoolRunner
from ambrs.camp import CampConfig
from netCDF4 import Dataset


In [2]:

# -----------------------------------------------------------
# Configure simulations and PPE (mirrors demo)
# -----------------------------------------------------------

# Ensemble and runtime parameters
n = 5          # number of ensemble members
n_part = 2000  # PartMC particles per run
dt = 60        # timestep [s]
nstep = 60     # total steps (=> 60 min run)

p0 = 101325    # reference pressure [Pa]
h0 = 500       # reference height [m]

# Executables (override as needed)
MAM4_EXE = "mam4"
PARTMC_EXE = "partmc"

# Root/output directories
repo_root = Path.cwd().resolve()
root_dir = repo_root / "runs"
ensemble_name = f"{int(dt*nstep/60)}min_{n_part}particles"
ensemble_dir = root_dir / ensemble_name
partmc_dir = ensemble_dir / "partmc_runs"
mam4_dir = ensemble_dir / "mam4_runs"
for d in [root_dir, ensemble_dir, partmc_dir, mam4_dir]:
    os.makedirs(d, exist_ok=True)

# -----------------------------------------------------------
# Define species (as in demo)
# -----------------------------------------------------------
so4 = ambrs.AerosolSpecies("SO4", molar_mass=96., density=1800., hygroscopicity=0.65)
pom = ambrs.AerosolSpecies("OC",  molar_mass=12.01, density=1000., hygroscopicity=0.001)
soa = ambrs.AerosolSpecies("MSA", molar_mass=40., density=2600., hygroscopicity=0.53)
bc  = ambrs.AerosolSpecies("BC",  molar_mass=12.01, density=1800., hygroscopicity=0.0)
dst = ambrs.AerosolSpecies("OIN", molar_mass=135.065, density=2600., hygroscopicity=0.1)
na  = ambrs.AerosolSpecies("Na",  molar_mass=23., density=2200., hygroscopicity=0.53)
cl  = ambrs.AerosolSpecies("Cl",  molar_mass=35.5, density=2200., hygroscopicity=0.53)
ncl = na
h2o = ambrs.AerosolSpecies("H2O", molar_mass=18., density=1000., ions_in_soln=1)

so2   = ambrs.GasSpecies("SO2",   molar_mass=64.07)
h2so4 = ambrs.GasSpecies("H2SO4", molar_mass=98.079)

# -----------------------------------------------------------
# Aerosol processes (baseline/native settings)
# -----------------------------------------------------------
processes = ambrs.AerosolProcesses(
    coagulation=True,
    condensation=True,
)

# -----------------------------------------------------------
# Ensemble specification (4 modes; same structure as demo)
# -----------------------------------------------------------
spec = ambrs.EnsembleSpecification(
    name=ensemble_name,
    aerosols=(so4, pom, soa, bc, dst, ncl, h2o),
    gases=(so2, h2so4),
    size=ambrs.AerosolModalSizeDistribution(modes=[
        # Mode 1: accumulation
        ambrs.AerosolModeDistribution(
            name="accumulation",
            species=[so4, pom, soa, bc, dst, ncl],
            number=stats.uniform(1e8, 1e11),
            geom_mean_diam=stats.loguniform(5e-8, 3e-7),
            log10_geom_std_dev=log10(1.8),
            mass_fractions=[
                stats.rv_discrete(values=([0.95], [1.])), # SO4
                stats.rv_discrete(values=([0.05], [1.])), # POM
                stats.rv_discrete(values=([0.0], [1.])),  # SOA placeholder
                stats.rv_discrete(values=([0.0], [1.])),  # BC
                stats.rv_discrete(values=([0.0], [1.])),  # DST
                stats.rv_discrete(values=([0.0], [1.])),  # NCL
            ],
        ),
        # Mode 2: aitken
        ambrs.AerosolModeDistribution(
            name="aitken",
            species=[so4, soa, ncl],
            number=stats.uniform(1e7, 1e11),
            geom_mean_diam=stats.rv_discrete(values=([2.6e-8], [1.])),
            log10_geom_std_dev=log10(1.6),
            mass_fractions=[
                stats.rv_discrete(values=([1.0], [1.])),  # SO4
                stats.rv_discrete(values=([0.0], [1.])),  # SOA
                stats.rv_discrete(values=([0.0], [1.])),  # NCL
            ],
        ),
        # Mode 3: coarse
        ambrs.AerosolModeDistribution(
            name="coarse",
            species=[dst, ncl, so4, bc, pom, soa],
            number=stats.uniform(1e6, 1e7),
            geom_mean_diam=stats.rv_discrete(values=([2e-6], [1.])),
            log10_geom_std_dev=log10(1.8),
            mass_fractions=[
                stats.rv_discrete(values=([0.0], [1.])),  # DST
                stats.rv_discrete(values=([0.0], [1.])),  # NCL
                stats.rv_discrete(values=([1.0], [1.])),  # SO4
                stats.rv_discrete(values=([0.0], [1.])),  # BC
                stats.rv_discrete(values=([0.0], [1.])),  # POM
                stats.rv_discrete(values=([0.0], [1.])),  # SOA
            ],
        ),
        # Mode 4: primary carbon
        ambrs.AerosolModeDistribution(
            name="primary carbon",
            species=[pom, bc],
            number=stats.rv_discrete(values=([0.], [1.])),
            geom_mean_diam=stats.loguniform(1e-8, 5e-8),
            log10_geom_std_dev=log10(1.8),
            mass_fractions=[
                stats.rv_discrete(values=([1.0], [1.])),  # POM
                stats.rv_discrete(values=([0.0], [1.])),  # BC
            ],
        ),
    ]),
    # gases: (SO2, H2SO4); keep SO2=0, H2SO4 ~ U
    gas_concs=tuple([stats.rv_discrete(values=([0.], [1.])), stats.uniform(1e-10, 1e-8)]),
    flux=stats.uniform(1e-11, 1e-8),  # placeholder (not used here)
    relative_humidity=stats.rv_discrete(values=([0.5], [1.])),
    temperature=stats.uniform(240, 70),
    pressure=p0,
    height=h0,
)

# Sample the ensemble (Latin Hypercube Sampling)
ensemble = ambrs.lhs(specification=spec, n=n)

# Scenario names (as in demo)
scenario_names = [str(ii).zfill(1) for ii in range(1, len(ensemble.flux)+1)]


In [3]:
from ambrs.camp import CampConfig, MechanismTemplate

# ~1 hour lifetime sink for H2SO4; SIMPOL params from your example
H2SO4_TAU_S = 3600.0
tmpl = MechanismTemplate(
    h2so4_first_order_k=1.0 / H2SO4_TAU_S,
    simpol_B=[0.0, -8.9, 0.0, 0.0],
    simpol_Nstar=1.23299521,
    aerosol_phase_name="mixed",
)

camp = CampConfig(
    lib_dir=None,           # auto: $CONDA_PREFIX/lib ; OR set to "/Users/fier887/miniforge3/envs/ambrs_camp_sync/lib"
    template=tmpl
)

# MAM4: turn OFF native condensation for the CAMP run
mam4_camp = ambrs.mam4.AerosolModel(
    processes=ambrs.AerosolProcesses(gas_phase_chemistry=True, condensation=False, coagulation=True),
    camp=camp,
)

# PartMC: turn OFF MOSAIC for the CAMP run
partmc_camp = ambrs.partmc.AerosolModel(
    processes=ambrs.AerosolProcesses(gas_phase_chemistry=False, condensation=False, coagulation=True),
    run_type="particle",
    n_part=n_part,
    n_repeat=1,
    camp=camp,
)


In [4]:

# -----------------------------------------------------------
# MAM4: baseline (native) and CAMP (native OFF)
# -----------------------------------------------------------

# Baseline
mam4_native = mam4.AerosolModel(
    processes=ambrs.AerosolProcesses(gas_phase_chemistry=True, condensation=True, coagulation=True),
    camp=None,
)
mam4_inputs = mam4_native.create_inputs(ensemble=ensemble, dt=dt, nstep=nstep)
mam4_runner = PoolRunner(
    model=mam4_native,
    executable=MAM4_EXE,
    root=str(mam4_dir),
    num_processes=1,
)
mam4_runner.run(mam4_inputs)

# CAMP
mam4_camp = mam4.AerosolModel(
    processes=ambrs.AerosolProcesses(gas_phase_chemistry=True, condensation=False, coagulation=True),
    camp=camp,
)
mam4_inputs_camp = mam4_camp.create_inputs(ensemble=ensemble, dt=dt, nstep=nstep)

# # Ensure per-scenario camp/ exists and write mechanism files
# for i, _ in enumerate(scenario_names):
#     scen_dir = mam4_dir / f"{i+1:0{len(scenario_names[0])}d}"
#     os.makedirs(scen_dir, exist_ok=True)
#     camp.write_common_files(str(scen_dir), scenario=None)

mam4_runner_camp = PoolRunner(
    model=mam4_camp,
    executable=MAM4_EXE,
    root=str(mam4_dir),
    num_processes=1,
)
mam4_runner_camp.run(mam4_inputs_camp)


mam4: one or more existing scenario directories found. Overwriting contents...
mam4: At least one run failed.
mam4: one or more existing scenario directories found. Overwriting contents...
mam4: At least one run failed.


[]

In [ ]:

# -----------------------------------------------------------
# PartMC: baseline (MOSAIC) and CAMP (MOSAIC OFF)
# -----------------------------------------------------------

partmc_native = partmc.AerosolModel(
    processes=ambrs.AerosolProcesses(gas_phase_chemistry=False, condensation=True, coagulation=True),
    run_type="particle",
    n_part=n_part,
    n_repeat=1,
    camp=None,
)
partmc_inputs = partmc_native.create_inputs(ensemble=ensemble, dt=dt, nstep=nstep)
partmc_runner = PoolRunner(
    model=partmc_native,
    executable=PARTMC_EXE,
    root=str(partmc_dir),
    num_processes=1,
)
partmc_runner.run(partmc_inputs)

partmc_camp = partmc.AerosolModel(
    processes=ambrs.AerosolProcesses(gas_phase_chemistry=False, condensation=False, coagulation=True),
    run_type="particle",
    n_part=n_part,
    n_repeat=1,
    camp=camp,
)
partmc_inputs_camp = partmc_camp.create_inputs(ensemble=ensemble, dt=dt, nstep=nstep)

# # Ensure per-scenario camp/ exists and write mechanism files
# for i, _ in enumerate(scenario_names):
#     scen_dir = partmc_dir / f"{i+1:0{len(scenario_names[0])}d}"
#     os.makedirs(scen_dir, exist_ok=True)
#     camp.write_common_files(str(scen_dir), scenario=None)

partmc_runner_camp = PoolRunner(
    model=partmc_camp,
    executable=PARTMC_EXE,
    root=str(partmc_dir),
    num_processes=1,
)
partmc_runner_camp.run(partmc_inputs_camp)


partmc: one or more existing scenario directories found. Overwriting contents...
partmc: At least one run failed.


AttributeError: 'CampConfig' object has no attribute 'write_common_files'

In [ ]:

# -----------------------------------------------------------
# Readers & checks (H2SO4, SO4, total-sulfur proxy)
# -----------------------------------------------------------

def read_mam4_timeseries(run_root: Path, scenario_name: str):
    nc = Dataset(str(run_root/scenario_name/'mam_output.nc'))
    out = {}
    out["time"] = np.array(nc.variables["time"][:]) if "time" in nc.variables else None
    if "h2so4_gas" in nc.variables:
        out["h2so4_gas"] = np.array(nc.variables["h2so4_gas"][:])
    else:
        cand = [v for v in nc.variables if "h2so4" in v.lower() and "gas" in v.lower()]
        out["h2so4_gas"] = np.array(nc.variables[cand[0]][:]) if cand else None
    so4 = None
    for v in nc.variables:
        vl = v.lower()
        if "so4" in vl and ("aer" in vl or "mode" in vl or "pm" in vl) and "gas" not in vl:
            arr = np.array(nc.variables[v][:])
            so4 = arr if so4 is None else so4 + arr
    out["so4_aer"] = so4
    return out

def read_partmc_timeseries(run_root: Path, scenario_name: str):
    outdir = run_root/scenario_name/"out"
    files = sorted([f for f in os.listdir(outdir) if f.endswith(".nc")])
    if not files: return {"time":None,"h2so4_gas":None,"so4_aer":None}
    times, h2so4_series = [], []
    so4_series = None
    for f in files:
        nc = Dataset(str(outdir/f))
        try:
            tstep = int(f.split("_")[-1].split(".")[0])
        except Exception:
            tstep = len(times)
        times.append(tstep)
        if "gas_species" in nc.variables and "gas_mixing_ratio" in nc.variables:
            names = nc.variables["gas_species"].names.split(",")
            data = np.array(nc.variables["gas_mixing_ratio"][:])
            if "H2SO4" in names:
                idx = names.index("H2SO4")
                h2so4_series.append(data[idx])
            else:
                h2so4_series.append(np.nan)
        if so4_series is None:
            so4_series = []
        found_so4 = False
        for v in nc.variables:
            vl = v.lower()
            if "so4" in vl and ("aer" in vl or "mass" in vl or "pm" in vl) and "gas" not in vl:
                arr = np.array(nc.variables[v][:])
                val = float(np.atleast_1d(arr)[-1]) if arr.size else np.nan
                so4_series.append(val); found_so4=True; break
        if not found_so4: so4_series.append(np.nan)
    return {"time": np.array(times),
            "h2so4_gas": np.array(h2so4_series) if h2so4_series else None,
            "so4_aer": np.array(so4_series) if so4_series is not None else None}

def rel_err(a, b):
    a = np.asarray(a); b = np.asarray(b)
    return np.abs(b - a) / np.maximum(1e-30, np.abs(a))

def summarize_pair(model_label, root_base: Path, root_camp: Path, scenario_name: str):
    r = {"model": model_label, "scenario": scenario_name}
    reader = read_mam4_timeseries if model_label.lower()=="mam4" else read_partmc_timeseries
    base = reader(root_base, scenario_name)
    camp_run = reader(root_camp, scenario_name)
    # H2SO4 similarity
    if base["h2so4_gas"] is not None and camp_run["h2so4_gas"] is not None:
        n = min(len(base["h2so4_gas"]), len(camp_run["h2so4_gas"]))
        r["h2so4_rel_err_mean"] = float(np.nanmean(rel_err(base["h2so4_gas"][:n], camp_run["h2so4_gas"][:n])))
        r["h2so4_base_final"] = float(base["h2so4_gas"][:n][-1]); r["h2so4_camp_final"] = float(camp_run["h2so4_gas"][:n][-1])
    else:
        r["h2so4_rel_err_mean"] = float("nan")
    # SO4 + total sulfur proxy
    if base["so4_aer"] is not None and camp_run["so4_aer"] is not None and base["h2so4_gas"] is not None and camp_run["h2so4_gas"] is not None:
        n2 = min(len(base["so4_aer"]), len(camp_run["so4_aer"]))
        r["so4_rel_err_mean"] = float(np.nanmean(rel_err(base["so4_aer"][:n2], camp_run["so4_aer"][:n2])))
        nb = min(n, n2)
        tb = np.asarray(base["h2so4_gas"][:nb]) + np.asarray(base["so4_aer"][:nb])
        tc = np.asarray(camp_run["h2so4_gas"][:nb]) + np.asarray(camp_run["so4_aer"][:nb])
        r["totalS_base_span"] = float(np.nanmax(tb) - np.nanmin(tb))
        r["totalS_camp_span"] = float(np.nanmax(tc) - np.nanmin(tc))
    else:
        r["so4_rel_err_mean"] = float("nan")
    return r


In [ ]:

# -----------------------------------------------------------
# Summaries across ensemble (no plotting)
# -----------------------------------------------------------
width = max(1, int(log10(len(scenario_names))) )  # width from names
results = []
for i, nm in enumerate(scenario_names):
    results.append(summarize_pair("MAM4", mam4_dir, mam4_dir, nm))     # baseline vs camp both under mam4_dir
for i, nm in enumerate(scenario_names):
    results.append(summarize_pair("PartMC", partmc_dir, partmc_dir, nm))

results


FileNotFoundError: [Errno 2] No such file or directory: '/Users/fier887/Library/CloudStorage/OneDrive-PNNL/Code/ambrs/runs/60min_2000particles/mam4_runs/1/mam_output.nc'